In [ ]:
from configs import models, env
from backend.llm.api import load_model, load_llm_langchain
from backend.prompts.templates import ehr_summarizer_prompt_template

config_loaded = {"model_config": models, "env": env}
llm = load_llm_langchain(source='groq', model_name='LLaMA-3', config=config_loaded)

[LLM Loader] Successfully initialized model 'llama-3.1-8b-instant' from 'groq'.


In [2]:
sample_ehr = """
The patient is a 68-year-old male with a history of hypertension, hyperlipidemia, and type 2 diabetes mellitus, admitted for chest pain.
ECG showed ST elevation; troponin elevated. Diagnosed with STEMI. Underwent PCI with stent placement.
Started on aspirin, atorvastatin, metoprolol, and lisinopril. Discharged in stable condition with follow-up in 1 week.
"""

prompt = ehr_summarizer_prompt_template.format(ehr_text=sample_ehr)
print("📋 Prompt preview:\n", prompt)

📋 Prompt preview:
 
You are a clinical summarization agent.

Given the following raw clinical note or discharge summary:

---

The patient is a 68-year-old male with a history of hypertension, hyperlipidemia, and type 2 diabetes mellitus, admitted for chest pain.
ECG showed ST elevation; troponin elevated. Diagnosed with STEMI. Underwent PCI with stent placement.
Started on aspirin, atorvastatin, metoprolol, and lisinopril. Discharged in stable condition with follow-up in 1 week.

---

Extract and return the following:
- Primary diagnosis
- Key findings (labs, vitals, procedures)
- Medications prescribed
- Follow-up recommendations
- Summary (in layman's terms)

Be clear and structured in your output.



In [3]:
response = llm.invoke(prompt)

print("📄 EHR Summary:\n")
print(response.content)


📄 EHR Summary:

**Clinical Summary**

### Patient Information

- **Age:** 68 years old
- **Gender:** Male

### Primary Diagnosis

- **STEMI (ST-Elevation Myocardial Infarction)**

### Key Findings

- **ECG:** ST elevation
- **Lab Results:** Elevated troponin
- **Procedures:** PCI (Percutaneous Coronary Intervention) with stent placement

### Medications Prescribed

- **Antiplatelet:** Aspirin
- **Statins:** Atorvastatin
- **Beta Blocker:** Metoprolol
- **ACE Inhibitor:** Lisinopril

### Follow-up Recommendations

- **Follow-up appointment:** In 1 week

### Summary (in layman's terms)

The patient had a heart attack (STEMI) and underwent a procedure to open up the blocked artery (PCI with stent placement). He was prescribed medications to prevent future heart problems and was discharged in stable condition. He needs to follow up with his doctor in 1 week.


In [5]:
import re

summary = {
    "diagnosis": re.search(r"Primary diagnosis:\s*(.+)", response.content, re.IGNORECASE),
    "medications": re.search(r"Medications prescribed:\s*(.+)", response.content, re.IGNORECASE),
    "follow_up": re.search(r"Follow-up recommendations:\s*(.+)", response.content, re.IGNORECASE),
}

for key, match in summary.items():
    print(f"{key.title()}: {match.group(1).strip() if match else '❓ Not found'}")

Diagnosis: ❓ Not found
Medications: ❓ Not found
Follow_Up: ❓ Not found
